In [1]:
import pandas as pd
import os
import re
from tqdm.auto import tqdm

In [2]:
cdli_dir = "../../../cdlidata/"

In [3]:
header = list(range(115))
files = os.listdir(cdli_dir)
files.sort()
list_ = []
for file in files:
    if file[:14] == "cdli_catalogue":
        df = pd.read_csv(cdli_dir + file, names = header, low_memory = False)
        list_.append(df)
df = pd.concat(list_)
df2 = df.iloc[:,0:63]
headers = list(df2.iloc[0,:])
df2.columns = headers
df2 = df2.iloc[1:,:]

In [4]:
keep = ['id_text', 'museum_no', 'primary_publication', 'accession_no']
df2 = df2[keep].fillna('')
df2

,id_text,museum_no,primary_publication,accession_no
1,1,VAT 01533,"CDLI Lexical 000002, ex. 065",
2,2,VAT 15263,"CDLI Lexical 000002, ex. 066",
3,3,VAT 15253,"ATU 3, pl. 081, W 9123,d",
4,4,VAT 15168,"CDLI Lexical 000002, ex. 051",
5,5,VAT 15153,"CDLI Lexical 000002, ex. 172",
...,...,...,...,...
160791,519789,BM 093503,"KASKAL 12, 001-030 12",
160792,519790,Grimme 53,"ZA 076, 081-084 + photo",
160793,519791,Anonymous 519791,unpublished unassigned,
160794,519792,Grimme 48,"ZA 076, 077-080 + photo",


In [20]:
df_tr = pd.read_csv('treasury_shoes2.txt', sep=',').fillna('')
len(df_tr.loc[df_tr.id_text == ''])

0

In [13]:
musno = dict(zip(df2['museum_no'], df2['id_text']))
musno = {re.sub(' 0+', ' ', m) : f"P{str(musno[m]).zfill(6)}" for m in musno}

In [14]:
for i in range(len(df_tr)):
    if df_tr.iloc[i]['id_text'] == '':
        df_tr.iloc[i]['id_text'] = musno.get(df_tr.iloc[i]['primary_publication'], '')
len(df_tr.loc[df_tr.id_text == ''])

73

In [15]:
pub = dict(zip(df2['primary_publication'], df2['id_text']))
pub = {re.sub('[,\.]', '', m) : f"P{str(pub[m]).zfill(6)}" for m in pub}
pub = {re.sub(' 0+', ' ', m).lower() : pub[m] for m in pub}

In [16]:
punct = '.,'
table = str.maketrans(dict.fromkeys(punct))

In [17]:
for i in range(len(df_tr)):
    if df_tr.iloc[i]['id_text'] == '':
        df_tr.iloc[i]['id_text'] = pub.get(df_tr.iloc[i]['primary_publication'].lower().translate(table), '')
len(df_tr.loc[df_tr.id_text == ''])

38

In [18]:
df_tr.loc[df_tr.id_text == '']

,primary_publication,Archive,id_text
52,SmCT 14,Shoe Archive,
53,StOr 9 13 32,Shoe Archive,
54,TCNY 209,Shoe Archive,
55,TENS 353,Shoe Archive,
58,TPTS 132,Shoe Archive,
61,AION 69 n. 1,Treasure Archive,
62,AnSt 33 p.74,Treasure Archive,
112,AUCT 2 326+336,Treasure Archive,
120,BJRL 64 p.111 n.68,Treasure Archive,
123,CDLJ 2002:1 n.8,Treasure Archive,


In [19]:
df_tr.to_csv('treasury_shoes2.txt', index=False, encoding = 'utf8')